In [63]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore', category = FutureWarning)

In [64]:
# # Import pandas libraries
# import pandas as pd

# # Load the dataset
# data = pd.read_csv("Data/activemq_result.csv", delimiter=',')
# # data = pd.read_csv("Data/avro_result.csv", delimiter=',')


# # Explore the dataset
# print(data.head()) 


In [65]:
# import pandas as pd
import glob

# Path to the directory containing your CSV files
file_path = "Train/*.csv"  # Use a wildcard to match all CSV files in the directory

# Get all file paths matching the pattern
all_files = glob.glob(file_path)

# Initialize an empty list to store DataFrames
data_list = []

# Read and combine files
for i, file in enumerate(all_files):
    if i == 0:
        # Include header only for the first file
        df = pd.read_csv(file, delimiter=',')
        print(df.shape)
    else:
        # Skip the header row for subsequent files
        df = pd.read_csv(file, delimiter=',', header=0)
        print(df.shape)
    data_list.append(df)

# Concatenate all DataFrames
combined_data = pd.concat(data_list, ignore_index=True)

data = combined_data.copy()

print(data.shape)

(4016, 42)
(619, 42)
(965, 42)
(12081, 42)
(1883, 42)
(6415, 42)
(2187, 42)
(2438, 42)
(473, 42)
(1873, 42)
(2238, 42)
(908, 42)
(1167, 42)
(629, 42)
(3936, 42)
(41828, 42)


In [66]:
# copy data
data_transform = data.copy()

In [67]:

# Convert 'bug-prone' column to 0 and 1
data_transform['bug-prone'] = data_transform['bug-prone'].apply(lambda x: 1 if x.strip() == 'bug-prone' else 0)

# Display the updated DataFrame
print(data_transform.head()) 


                                         Method name  C20  C3  C4  C1  C5  C6  \
0  org.apache.activemq.transport.amqp.AmqpFramePa...    4  10   1   9   2   5   
1  org.apache.activemq.transport.amqp.AmqpHeader....    5   6   0   6   1   3   
2  org.apache.activemq.transport.amqp.AmqpHeader....    1  13   0  13   3   9   
3  org.apache.activemq.transport.amqp.AmqpInactiv...    1   5   0   5   1   3   
4  org.apache.activemq.transport.amqp.AmqpInactiv...    6   9   0   9   1   5   

   C2  C21  C18  ...  H4  H1  H2  H3       H12       H13       H14   H15  H5  \
0   0    2    4  ...   1   2   2   4  0.200000  0.200000  1.000000   4.0   1   
1   0    5    2  ...   1   1   3   4  0.333333  1.000000  0.333333   4.0   1   
2   0    9    3  ...   1  10   0  10  0.769231  0.000000  0.000000  10.0   1   
3   0    3    1  ...   1   2   0   2  0.400000  0.000000  0.000000   2.0   1   
4   0    4    3  ...   1   3   2   5  0.333333  0.222222  1.500000   5.0   1   

   bug-prone  
0          0  
1 

In [68]:

# split feature data and target data
feature_X = data_transform.drop(columns=['Method name','bug-prone'])
y = data_transform['bug-prone']



In [69]:
from imblearn.combine import SMOTETomek
from sklearn.preprocessing import StandardScaler

# Apply SMOTETomek first (on original feature space)
smote_tomek = SMOTETomek(random_state=42)
X_resampled, y_resampled = smote_tomek.fit_resample(feature_X, y)

# Apply StandardScaler after resampling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_resampled)

X = X_scaled
y = y_resampled

In [70]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from catboost import CatBoostClassifier

models = {
    
    "RandomForest" :  RandomForestClassifier(n_estimators=500, max_depth=None, min_samples_split=5, 
                                       min_samples_leaf=2, class_weight='balanced', random_state=42),
    "AdaBoostClassifier" : AdaBoostClassifier(n_estimators=100, learning_rate=0.8, random_state=42),
    "BaggingClassifier" : BaggingClassifier(n_estimators=100, max_samples=0.8, random_state=42),
    "KNeighborsClassifier" : KNeighborsClassifier(n_neighbors=3, weights='distance', metric='manhattan'),
    "MLPClassifier" : MLPClassifier(activation='relu', hidden_layer_sizes=(200,100), max_iter=2000, 
                               learning_rate='adaptive', random_state=42),
    "HistGradientBoostingClassifier" : HistGradientBoostingClassifier(random_state=42),   
    "DecisionTreeClassifier" : DecisionTreeClassifier(random_state=42),
    "SVC" : SVC(random_state=42, probability=True, C=10, kernel='poly', gamma='scale'),
    "XGBoost": XGBClassifier(n_estimators=200, learning_rate=0.05, max_depth=4, random_state=42),
    "CatBoost": CatBoostClassifier(iterations=200, learning_rate=0.05, depth=4, verbose=0, random_state=42)
}




In [71]:
import numpy as np
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score
)




In [72]:
# Import pandas libraries
import pandas as pd

feature_names = feature_X.columns

X_df = pd.DataFrame(X, columns=feature_names)

Method 1

In [73]:

# avg_probabilities = []
# avg_scores = []

# # Initialize cross-validation
# kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# # Train all models and get probabilities
# for name, model in models.items():
#     print(name)
#     probabilities = []
#     max_scores = []

#     for train_index, test_index in kf.split(X_df, y):
#         X_train, X_test = X_df.iloc[train_index], X_df.iloc[test_index]
#         y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
#         model.fit(X_train, y_train)
#         y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probability for class 1
        
#         probabilities.append(y_pred_proba)
#         max_scores.append(roc_auc_score(y_test, y_pred_proba))  # Use AUC as weight

#     probabilities = np.concatenate(probabilities)  # Ensures consistent shape
#     avg_probabilities.append(np.mean(probabilities))
#     avg_scores.append(np.mean(max_scores))
#     print(avg_probabilities)
#     print(avg_scores)







In [74]:
# # Normalize AUC scores to get weights
# weights = [score / sum(avg_scores) for score in avg_scores]
# print(weights)

# combined_probs = [weight * probs for weight, probs in zip(weights, y)]
# print(len(combined_probs))

# # Determine best threshold using precision-recall
# from sklearn.metrics import precision_recall_curve

# def get_best_threshold_precision_recall(y_true, y_proba):
#     precision, recall, thresholds = precision_recall_curve(y_true, y_proba)
#     f1_scores = np.where((precision + recall) == 0, 0, 2 * (precision * recall) / (precision + recall))
#     best_threshold = thresholds[np.argmax(f1_scores)]
#     return best_threshold

# best_threshold_pr = get_best_threshold_precision_recall(y, combined_probs)
# print(best_threshold_pr)

Method 2

In [75]:
# from sklearn.metrics import (
#     make_scorer, precision_score, recall_score, f1_score, 
#     matthews_corrcoef, roc_auc_score
# )
# from sklearn.model_selection import cross_val_score, cross_validate

# # Define custom scorers for cross-validation
# scoring = {
#     'accuracy': 'accuracy',
#     'precision': make_scorer(precision_score, average='binary'),  # For binary classification
#     'recall': make_scorer(recall_score, average='binary'),
#     'f1': make_scorer(f1_score, average='binary'),
#     'mcc': make_scorer(matthews_corrcoef),
#     'auc': make_scorer(roc_auc_score) # , needs_proba=True
# }

# # Helper function to display results
# def print_cv_results(results):
#     for metric in scoring.keys():
#         mean = results[f'test_{metric}'].mean()
#         std = results[f'test_{metric}'].std()
#         print(f"{metric.capitalize()}: {mean:.4f} ± {std:.4f}")

# def crossvalidate_fun(classifier, X_train, y_train):
#     cv_results = cross_validate(classifier, X_train, y_train, cv=10, scoring=scoring)
#     print_cv_results(cv_results)

# scores = []

# # Train all models and get probabilities
# for name, model in models.items():
#     cv_results = cross_validate(model, X, y, cv=10, scoring=scoring)
#     scores.append(cv_results['test_auc'].mean())

# print(scores)


Method 3

In [76]:
# Import train_test_split library
from sklearn.model_selection import train_test_split

# Split the data into training (80%) and validation sets (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [77]:

probabilities = []
max_scores = []

# Train all models and get probabilities
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probability for class 1
    probabilities.append(y_pred_proba)
    max_scores.append(roc_auc_score(y_test, y_pred_proba))  # Use AUC as weight

# Normalize AUC scores to get weights
weights = [score / sum(max_scores) for score in max_scores]

combined_probs = np.sum([weight * probs for weight, probs in zip(weights, probabilities)], axis=0)

# Determine best threshold using precision-recall
from sklearn.metrics import precision_recall_curve

def get_best_threshold_precision_recall(y_true, y_proba):
    precision, recall, thresholds = precision_recall_curve(y_true, y_proba)
    f1_scores = np.where((precision + recall) == 0, 0, 2 * (precision * recall) / (precision + recall))   
    best_threshold = thresholds[np.argmax(f1_scores)]
    return best_threshold

best_threshold_pr = get_best_threshold_precision_recall(y_test, combined_probs)

# Convert probabilities to binary predictions using threshold
predictions = (combined_probs > best_threshold_pr).astype(int)


# Evaluate combined results
print("\nFinal Combined Results:")
print(f"Accuracy: {accuracy_score(y_test, predictions):.4f}")
print(f"Precision: {precision_score(y_test, predictions):.4f}")
print(f"Recall: {recall_score(y_test, predictions):.4f}")
print(f"F1-Score: {f1_score(y_test, predictions):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, predictions):.4f}")
print(f"AUC: {roc_auc_score(y_test, combined_probs):.4f}")





Final Combined Results:
Accuracy: 0.8536
Precision: 0.8436
Recall: 0.8728
F1-Score: 0.8580
MCC: 0.7075
AUC: 0.9375


Save Models, Weights, and Threshold

In [79]:
import pickle

# Save all models
for name, model in models.items():
    with open(f"Models/{name}_model.pkl", "wb") as f:
        pickle.dump(model, f)

# Save weights and threshold
with open("weights_threshold.pkl", "wb") as f:
    pickle.dump({"weights": weights, "best_threshold_pr": best_threshold_pr}, f)


Load & Use Later (Without Training Again)

In [80]:
# Load models
loaded_models = {}
for name in models.keys():
    with open(f"Models/{name}_model.pkl", "rb") as f:
        loaded_models[name] = pickle.load(f)

# Load weights and threshold
with open("weights_threshold.pkl", "rb") as f:
    saved_data = pickle.load(f)
    weights = saved_data["weights"]
    best_threshold_pr = saved_data["best_threshold_pr"]


Testing

In [81]:
# Import pandas libraries
import pandas as pd

# Load the dataset
trdata = pd.read_csv("Test/zookeeper_result.csv", delimiter=',')


# Explore the dataset
print(trdata.head()) 

                                         Method name  C20  C3  C4  C1  C5  C6  \
0        org.apache.zookeeper.graph.Log4JSource.init    8  52   5  42  10  33   
1        org.apache.zookeeper.graph.Log4JSource.main    3  48   7  37  14  31   
2  org.apache.zookeeper.graph.RandomAccessFileRea...    2  30   1  29   5  23   
3  org.apache.zookeeper.graph.servlets.NumEvents....    3  35   6  29   8  24   
4  org.apache.zookeeper.graph.servlets.StaticCont...    4  20   1  18   3  12   

   C2  C21  C18  ...  H4  H1  H2  H3       H12  H13  H14  H15  H5  \
0   5   19   32  ...   1   1   0   1  0.019231  0.0  0.0  1.0   1   
1   4    7    2  ...   1   2   0   2  0.041667  0.0  0.0  2.0   1   
2   0    7   16  ...   1   1   0   1  0.033333  0.0  0.0  1.0   1   
3   0   11   12  ...   1   1   0   1  0.028571  0.0  0.0  1.0   1   
4   1   12    3  ...   1   3   0   3  0.150000  0.0  0.0  3.0   1   

       bug-prone  
0  not bug-prone  
1      bug-prone  
2      bug-prone  
3      bug-prone  
4  

In [82]:
# split feature data and target data
trdata_X = trdata.drop(columns=['Method name','bug-prone'])

trdata_X_scaled = scaler.fit_transform(trdata_X)

In [83]:

tr_probabilities = []


# Train all models and get probabilities
for name, model in loaded_models.items():
    y_pred_proba = model.predict_proba(trdata_X_scaled)[:, 1]  # Probability for class 1
    tr_probabilities.append(y_pred_proba)


tr_combined_probs = np.sum([weight * probs for weight, probs in zip(weights, tr_probabilities)], axis=0)



# Convert probabilities to binary predictions using threshold
tr_predictions = (tr_combined_probs > best_threshold_pr).astype(int)




In [84]:
trdata_y = trdata['bug-prone'].apply(lambda x: 1 if x.strip() == 'bug-prone' else 0)

# Evaluate combined results
print("\nFinal Combined Results:")
print(f"Accuracy: {accuracy_score(trdata_y, tr_predictions):.4f}")
print(f"Precision: {precision_score(trdata_y, tr_predictions):.4f}")
print(f"Recall: {recall_score(trdata_y, tr_predictions):.4f}")
print(f"F1-Score: {f1_score(trdata_y, tr_predictions):.4f}")
print(f"MCC: {matthews_corrcoef(trdata_y, tr_predictions):.4f}")
print(f"AUC: {roc_auc_score(trdata_y, tr_combined_probs):.4f}")





Final Combined Results:
Accuracy: 0.5944
Precision: 0.5806
Recall: 0.8793
F1-Score: 0.6994
MCC: 0.1837
AUC: 0.7233


In [85]:
index = 12

if tr_predictions[index] == 1:
    print("buggy")
else:
    print("not buggy")

buggy
